In [1]:
from googlesearch import search 
import import_ipynb
from bs4 import BeautifulSoup as bsoup
from multiprocessing.dummy import Pool
import requests
import sys
from requests.exceptions import RequestException
from contextlib import closing
import re
import json
import os

In [2]:
def content_Preprocess(content):
    if content is not None:
        title=''
        text=''
        soup = bsoup(content, 'html.parser')
        try:
            title += soup.find('title').getText()
        except AttributeError as error:
            print(error)
        for script in soup(["script", "style"]):
            script.extract()    # rip it out
        # get text
        text += soup.get_text()
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        return (title, text)
    else:
        return None

In [3]:
def simple_get(url):
    try:
        with closing(requests.get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    print(e)

In [4]:
def load_json(filename, directory='./tmp/'):
    path=os.path.join(directory, filename + "." + 'json')
    with open(path) as f:
        data = json.load(f)
    return data

In [15]:
def get_colleage(name):
    ent_name=name.replace(' +',"_")
    content=simple_get('http://en.wikipedia.org/wiki/'+ent_name)
    if content is not None:
        soup = bsoup(content, 'html.parser')
        table = soup.find('table', attrs={'class':'infobox biography vcard'})
        if table is not None:
            table_body = table.find('tbody')
            rows = table_body.find_all('tr')
            for row in rows:
                col_hd = row.find('th')
                col=row.find('td')
                if col_hd is not None and col is not None:
                    col = col.text.strip()
                    col_hd=col_hd.text.strip()
                    if "Alma" in col_hd:
                        return (name,col.split("\n")[-1])

In [191]:
def multiple_process(ent_name, get_colleage, proc=2):
    p=Pool(proc)
    try:
        cllg=p.map(get_colleage, ent_name)
    except Exception as e:
        print(e)
    return cllg

In [ ]:
ent_name=[]
pid_name_entid=load_json('pid2ent2','./')
for a in pid_name_entid:
    ent_name.append(a[1])
a=multiple_process(ent_name, get_colleage)

In [ ]:
"^(?=.*?\b[A|a]lan\b)(?=.*?\b[T|t]uring\b)(?=.*?\b[D|d]octorate\b).*$"

In [ ]:
query = 'Alan turing AND doctorate institute'
repeat=[]
for url in search(query, stop=20):
    if 'YouTube' not in url:
        if not any((a in url or url in a) for a in repeat):
            r=simple_get(url)
            if content_Preprocess(r) is not None:
                title_text=content_Preprocess(r)
                print(url)
                match=re.search("([Aa]lan|[Tt]uring).*([Dd]octorate).*?\.",title_text[1])
                if match is not None:
                    print(match.group())
                repeat.append(url)
            else:
                continue
        else:
            continue

In [9]:
a=requests.get('http://www.openthesis.org/search/search.html?from=advancedSearch&queryString=AU%28%22Miller%22%29&searchButton.x=76&searchButton.y=26&sortOrder=Relevance&fromDate=&toDate=')

In [10]:
a.content

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n    <head>\n        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>\n        <title>Document Search Results - OpenThesis</title>\n        <link rel=\'icon\' href=\'/images/favicon__v28221.ico\' type=\'image/vnd.microsoft.icon\'/>\n        <link rel=\'stylesheet\' type=\'text/css\' href=\'/css/style__v28221.gz.css\' />\n        \n        \n        \n            \n        \n                  \n        <meta name="layout" content="main"/>\n        \n\n    </head>\n    <body>\n        <div id="container">\n            \n                <div class="topSection line">\n    <div class="topSectionLeft">\n    \n    <div id="logo">\n        <a href="/">Upload and Search Theses and Dissertations - OpenThesis</a>\n    </div>\n\n     \n\n</div>\n\n    <div>\n\n          <!--[if lt IE 7]>\n<style>\n\t#userMenu{\n    margin-b

In [11]:
soup = bsoup(a.content, 'html.parser')

In [13]:
print(soup.text)





Document Search Results - OpenThesis









Upload and Search Theses and Dissertations - OpenThesis





Login
Create Free Account





















Home
Upload
Search
FAQ
For Universities
For Authors
Help





Search Results
                    
                        Back to Last Search



                                
                                    Showing 1 - 50 of 617 results
                                
                                for query 
                                
                                    AU("Miller")

12345678910..13Next >




Match
Authors
Date
School
Type


On reductionism ontological, scientific, and biological issues in reductionism /


1
1969- III, Miller

The University of Georgia - USA - Georgia
THESIS


Accurate measurements of the collision stopping powers for 5 to 30 MeV electrons


2
MacPherson, Miller S.
1998-01-01

THESIS


Avaliação do sêmen caprino diluído em citrato-gema, resfriado e armazenado a 5ºC por 24 horas
Ava